In [1]:
from pathlib import Path

In [2]:
if not Path("input.txt").exists():
    # We always start with a dataset to train on. Let's download the tiny shakespeare dataset
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [3]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [5]:
# let's look at the first 500 characters
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [6]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# We'll use character level tokenization, but in practice people use sub-word level tokenizers like tiktoken or SentencePiece
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8 # context length
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
decode(train_data[:block_size + 1].tolist())

'First Cit'

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]  # offset by 1
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [13]:
torch.manual_seed(1337)
batch_size = 4  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?


def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(high=len(data) - block_size, size=(batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("----")

for b in range(batch_size):  # batch dimension
    for t in range(block_size):  # time dimension
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [14]:
print(xb)  # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits: torch.Tensor = self.token_embedding_table(idx)  # (B, T, C)

        # problem: F.cross_entropy expects (B, C, T)

        # my solution
        # loss = F.cross_entropy(logits.transpose(1, 2), targets)

        # A.K.'s solution
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # predict
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T + 1)
        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [16]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [17]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [18]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.5727508068084717


In [19]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht 


### The mathematical trick in self-attention

In [20]:
# consider the following toy example:

torch.manual_seed(1337)
B, T, C = 4, 8, 2  # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [21]:
# "the simplest way for a token to communicate with the previous tokens is to do an average of all the preceding elements"
# "if I am token 5, I take my features and also of the channels of the previous tokens. But  that is very lossy"

In [22]:
# This is very inefficient
# We want x[b, t] = mean_{i<=t} x[b, i]
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, : t + 1]  # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [23]:
# Getting closer to the trick

torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print("a=")
print(a)
print("---")
print("b=")
print(b)
print("---")
print("c=")
print(c)
print("---")

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])
---


In [24]:
torch.tril(torch.ones(3, 3))  # triangular

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [25]:
c = torch.tril(a) @ b
print("a=")
print(a)
print("---")
print("b=")
print(b)
print("---")
print("c=")
print(c)
print("---")

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])
---


In [26]:
a = torch.tril(a)
a = a / torch.sum(a, 1, keepdim=True)

c = a @ b
print("a=")
print(a)
print("---")
print("b=")
print(b)
print("---")
print("c=")
print(c)
print("---")

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
---


We got the averages!!

In [27]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x  # (T, T) @ (B, T, C) --> broadcast (B, T, T) @ (B, T, C) --> (B, T, C)
torch.allclose(xbow, xbow2, rtol = 0.0001)

True

**The trick is that we used batch matrix multiply to do this weighted aggregation. We're doing weighted sums in a triangular form**

In [28]:
# One more version to achieving the above: using softmax
tril = torch.tril(torch.ones(T, T))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [29]:
wei = torch.zeros((T, T))
wei

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [30]:
wei = wei.masked_fill(tril == 0, float("-inf"))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [31]:
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

**Amazing!!!** <br/>
The reason why this is more interesting and is used in self-attention is because the weights in `wei` are an "interaction strength" <br/>
i.e. how much from each token from the past do we want to aggregate and average up (instead of equal contributions) <br/>
doing `-inf` means tokens from the future cannot communicate, `e**-inf = 0`

You can do weighted aggregations of your past elements by using matrix multiplication of a lower triangular fashion. The elements tell you how much of element fuses into this position

### version 4: self-attention!

In [32]:
# self attention solves that wei is data dependent!
# every single token will emit two vectors: Q and K
# Q: what am I looking for
# K: what do I contain
# ==> Q dot products with K ==> wei
# eg. if Q and K are aligned, they will give a high number

In [33]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32  # batch, time, channels
x = torch.randn(B, T, C)

# let's see a single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)  # (B, T, 16)
q = query(x)  # (B, T, 16)
# scaled (1/sqrt(head size)) to not make softmax blow up
wei = q @ k.transpose(-2, -1) * head_size ** -0.5  # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
# out = wei @ x
out.shape

torch.Size([4, 8, 16])

In [34]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3966, 0.6034, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3069, 0.2892, 0.4039, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3233, 0.2175, 0.2443, 0.2149, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1479, 0.2034, 0.1663, 0.1455, 0.3369, 0.0000, 0.0000, 0.0000],
        [0.1259, 0.2490, 0.1324, 0.1062, 0.3141, 0.0724, 0.0000, 0.0000],
        [0.1598, 0.1990, 0.1140, 0.1125, 0.1418, 0.1669, 0.1061, 0.0000],
        [0.0845, 0.1197, 0.1078, 0.1537, 0.1086, 0.1146, 0.1558, 0.1553]],
       grad_fn=<SelectBackward0>)

In [35]:
# x is like private info to the token
# v is "if you find me interesting, here's what I will communicate to you"

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

### Layer Norm

In [36]:
class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        # parameters (trained with backprop)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        # buffers (trained with a running 'momentum update')
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)

    def __call__(self, x: torch.Tensor):
        # calculate the forward pass
        if self.training:
            xmean = x.mean(0, keepdim=True)  # batch mean
            xvar = x.var(0, keepdim=True)  # batch variance
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)  # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        # update the buffers
        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100)  # batch size 32 of 100 dim vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [37]:
x[:, 0].mean(), x[:, 0].std()  # mean,std of one feature across all batch inputs

(tensor(7.4506e-09), tensor(1.0000))

In [38]:
x[0, :].mean(), x[0, :].std()  # mean,std of a single input from the batch, of its features

(tensor(0.0411), tensor(1.0431))

**Batch norm to layer norm: normalize the rows, not the columns (dim=1 instead of 0)**

In [43]:
class LayerNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        # parameters (trained with backprop)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x: torch.Tensor):
        # calculate the forward pass
        xmean = x.mean(1, keepdim=True)  # batch mean
        xvar = x.var(1, keepdim=True)  # batch variance

        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)  # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100)  # batch size 32 of 100 dim vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [44]:
x[:, 0].mean(), x[:, 0].std()  # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [45]:
x[0, :].mean(), x[0, :].std()  # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

### Encoder

In [ ]:
# French to English translation example:

# <---------- ENCODE -------------------><---------------------- DECODE ------------>
#  les reseaux de neurones sont geniaux!  <START> neural networks are awesome! <END>

- In our transformer (`v2.py`), we use the decoder only. We're generating text not conditioned on anything. Using the triangular mask makes it a decoder
- The original paper has encoder and decoder because it focuses on machine translation.
- it expects some tokens from the original language to be encoded, and it is expected to decode the translation in the target language
- You read the original text and then the new special token (`<START>` in this case) and output tokens, ending with the `<END>` token
- The generation (decoder part) is conditioned on additional information which is the encoded original text.
- Encoded task doesn't have mask, all tokens are free to talk to each other
- Encoding is brought into the decoder through the "cross attention"
    - Queries are still generated from X (decoder), but K and V are from the encoder.
    - This conditions the text generation not only on the previous generated words but also on the encoded original text

# Assignment

In [39]:
import torch
from torch import nn
from torch.functional import F

n_embd = 384
n_head = 6
dropout = 0.2
batch_size = 3
block_size = 4

In [63]:
class Head(nn.Module):
    """one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)  # (B, T, C)
        q = self.query(x)  # (B, T, C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5  # (B, T, C) @ (B, C, T) ---> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B, T, C)
        out = wei @ v  # (B, T, T) @ (B, T, C) --> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class ParallelMultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.num_heads = num_heads
        self.key = nn.Linear(n_embd, num_heads * head_size, bias=False)
        self.query = nn.Linear(n_embd, num_heads * head_size, bias=False)
        self.value = nn.Linear(n_embd, num_heads * head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout1 = nn.Dropout(dropout)
        # self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embd)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k: torch.Tensor = self.key(x)  # (B, T, C * num_heads) I think
        k = k.view(B, self.num_heads, T, -1)  # (B, num_heads, T, C) I think
        # k = k.view(B, T, -1, C)  # (B, T, num_heads, C) I think
        # k = k.transpose(1, 2)  # (B, num_heads, T, C) I think
        q: torch.Tensor = self.query(x)  # (B, T, C * num_heads) I think
        q = q.view(B, self.num_heads, T, -1)  # (B, num_heads, T, C) I think
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5  # (B, num_heads, T, C) I think @ (B, num_heads, C, T) ---> (B, num_heads, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, num_heads, T, T) I think
        wei = F.softmax(wei, dim=-1)  # (B, num_heads, T, T) I think
        wei = self.dropout1(wei)
        # perform the weighted aggregation of the values
        v: torch.Tensor = self.value(x)  # (B, T, C * num_heads) I think
        v = v.view(B, self.num_heads, T, -1)  # (B, num_heads, T, C) I think
        
        out = wei @ v  # (B, num_heads, T, T) I think @ (B, num_heads, T, C) I think --> (B, num_heads, T, C) I think

        # out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = out.view(B, T, -1)
        out = self.proj(out)
        out = self.dropout2(out)
        return out


In [64]:
model = ParallelMultiHeadAttention(n_head, n_embd // n_head)
n_head, n_embd // n_head

(6, 64)

In [65]:
x = torch.randn((batch_size, block_size, n_embd))
x.shape

torch.Size([3, 4, 384])

In [66]:
x

tensor([[[-0.9313, -2.4785,  2.3419,  ...,  0.5490,  0.6963,  0.3184],
         [ 0.9191, -1.1399, -0.0377,  ...,  1.1271,  1.9886,  0.3397],
         [-0.7822,  1.9243,  0.2267,  ...,  1.0478,  0.3675, -1.2472],
         [-0.3277,  0.1099, -1.2866,  ...,  0.8012,  1.5628,  0.2056]],

        [[ 1.5041,  0.5908,  0.6024,  ...,  1.9650,  1.6664,  0.8255],
         [ 0.2701,  0.0767, -0.1388,  ...,  0.0940, -0.3641, -1.4299],
         [-0.4509, -1.0135, -1.2111,  ...,  0.5365, -0.5779,  1.1980],
         [ 0.3749,  0.6322, -1.7075,  ..., -1.2488, -1.7940,  0.3149]],

        [[-1.0187,  1.1852,  1.5007,  ..., -0.1047,  0.0277, -0.0165],
         [ 1.3908,  2.0922, -1.1514,  ...,  0.0876, -0.5412, -0.7248],
         [ 0.2610,  0.2616, -0.6810,  ..., -1.6476,  0.4136,  1.9287],
         [ 0.1398,  0.5695,  0.4683,  ..., -1.3479, -0.1842,  1.8200]]])

In [67]:
out = model(x)
out

tensor([[[ 0.3926,  0.1074,  0.2722,  ..., -0.0000,  0.7281, -0.2143],
         [-0.4196,  0.0000,  0.1722,  ...,  0.0721, -0.7958, -0.4620],
         [-0.1253, -0.2069, -0.0340,  ...,  0.5302, -0.3135,  0.0775],
         [ 0.0000, -0.3550,  0.4113,  ...,  0.0905,  0.3060, -0.0605]],

        [[ 0.0000, -0.0530, -0.2312,  ..., -1.1856, -0.1850, -0.0000],
         [-0.2109,  0.0045,  0.2274,  ..., -0.1008,  0.1083, -0.2098],
         [ 0.0000,  0.5153, -0.0000,  ...,  0.0631, -0.1785,  1.1498],
         [ 0.4003,  0.0102, -0.4305,  ...,  0.3990, -0.3958, -0.1806]],

        [[-0.1071, -0.2941,  0.0556,  ...,  0.2443,  0.4236,  0.2230],
         [ 0.1055, -0.1496,  0.5677,  ...,  0.7670, -0.3646,  0.0000],
         [ 0.4953, -0.2046,  0.2270,  ...,  0.0345, -0.4220, -0.6558],
         [ 0.3800, -0.5918,  0.2712,  ..., -0.0000, -0.4474,  0.1926]]],
       grad_fn=<MulBackward0>)

In [68]:
out.shape

torch.Size([3, 4, 384])

In [25]:
rand1 = torch.randn((2, 3, 4))
rand2 = torch.randn((2, 3, 4, 4))
rand1

tensor([[[-0.4708,  0.8636,  1.1711, -0.1806],
         [ 1.3131,  0.2736, -0.2998, -1.2926],
         [-0.3387,  0.6141,  1.0641,  0.8745]],

        [[-1.1395,  1.5234,  0.9562, -0.3741],
         [ 0.5747, -0.8092,  0.4757,  0.2882],
         [ 0.0070,  0.0371, -1.3711, -0.0125]]])

In [26]:
rand2

tensor([[[[ 1.4687, -0.5588,  0.6840, -0.4209],
          [-0.1142, -0.5692, -0.3949,  1.6069],
          [-1.5965, -0.1054, -0.3229, -0.5735],
          [-0.5919, -0.7921,  0.2347,  0.0441]],

         [[-1.1795,  0.8806,  2.2596, -1.3994],
          [-0.1292, -0.5375,  0.9403,  0.4551],
          [ 0.3646,  0.4014, -1.2652,  1.1791],
          [-0.1834, -0.1759,  1.5784,  0.0272]],

         [[-0.0887,  0.2999,  0.2502,  1.7511],
          [-0.1558,  1.2821,  1.6907, -1.3307],
          [ 0.4512, -0.2160, -1.0190, -0.2122],
          [ 0.1815, -0.1083, -2.0826,  0.0931]]],


        [[[-2.8806, -0.2414, -0.4196, -0.7122],
          [ 0.0818, -1.2383, -0.3906,  0.8633],
          [-0.9123, -0.1638,  0.7136,  0.7332],
          [ 0.1743,  0.2909,  1.2049,  0.0336]],

         [[ 1.7113,  1.1373, -0.2199, -0.1475],
          [ 0.4940,  0.0667, -0.5507, -0.3421],
          [ 1.4832, -0.1310, -1.0491,  1.3679],
          [ 1.0069, -0.1520, -0.5712,  1.4798]],

         [[ 0.4383, -1.0750,

In [27]:
tril = torch.tril(torch.ones(block_size, block_size))
tril

tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])

In [28]:
rand2.masked_fill(tril[:block_size, :block_size] == 0, float("-inf"))

tensor([[[[ 1.4687,    -inf,    -inf,    -inf],
          [-0.1142, -0.5692,    -inf,    -inf],
          [-1.5965, -0.1054, -0.3229,    -inf],
          [-0.5919, -0.7921,  0.2347,  0.0441]],

         [[-1.1795,    -inf,    -inf,    -inf],
          [-0.1292, -0.5375,    -inf,    -inf],
          [ 0.3646,  0.4014, -1.2652,    -inf],
          [-0.1834, -0.1759,  1.5784,  0.0272]],

         [[-0.0887,    -inf,    -inf,    -inf],
          [-0.1558,  1.2821,    -inf,    -inf],
          [ 0.4512, -0.2160, -1.0190,    -inf],
          [ 0.1815, -0.1083, -2.0826,  0.0931]]],


        [[[-2.8806,    -inf,    -inf,    -inf],
          [ 0.0818, -1.2383,    -inf,    -inf],
          [-0.9123, -0.1638,  0.7136,    -inf],
          [ 0.1743,  0.2909,  1.2049,  0.0336]],

         [[ 1.7113,    -inf,    -inf,    -inf],
          [ 0.4940,  0.0667,    -inf,    -inf],
          [ 1.4832, -0.1310, -1.0491,    -inf],
          [ 1.0069, -0.1520, -0.5712,  1.4798]],

         [[ 0.4383,    -inf,

In [9]:
rand1.view(2, 3, 2, 2)

tensor([[[[ 0.2154,  0.0628],
          [ 1.6783, -0.8057]],

         [[ 1.3895, -0.3504],
          [ 0.0886, -0.1807]],

         [[ 0.5660, -0.3521],
          [ 0.5289,  0.1716]]],


        [[[ 0.0070, -1.2198],
          [ 0.3417, -0.2371]],

         [[ 0.2941,  0.8191],
          [ 1.5681, -0.3443]],

         [[-0.0808,  0.9674],
          [-1.4946,  1.3086]]]])

In [10]:
rand1.view(2, 2, 3, 2)

tensor([[[[ 0.2154,  0.0628],
          [ 1.6783, -0.8057],
          [ 1.3895, -0.3504]],

         [[ 0.0886, -0.1807],
          [ 0.5660, -0.3521],
          [ 0.5289,  0.1716]]],


        [[[ 0.0070, -1.2198],
          [ 0.3417, -0.2371],
          [ 0.2941,  0.8191]],

         [[ 1.5681, -0.3443],
          [-0.0808,  0.9674],
          [-1.4946,  1.3086]]]])